The new data model currently lives in the branch `data-model`. You will have to install that version of PMD, which you can do by executing the code below.

In [1]:
]add /Users/sclaeys/code/PowerModelsDistribution.jl#data-model

  Updating registry at `~/.julia/registries/General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`
  Updating git-repo `/Users/sclaeys/code/PowerModelsDistribution.jl`
 Resolving package versions...
  Updating `~/.julia/environments/v1.0/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.0/Manifest.toml`
 [no changes]


In [2]:
using PowerModelsDistribution

┌ Info: Recompiling stale cache file /Users/sclaeys/.julia/compiled/v1.0/PowerModelsDistribution/utAoY.ji for PowerModelsDistribution [d7431456-977f-11e9-2de3-97ff7677985e]
└ @ Base loading.jl:1190
┌ Warning: Replacing docs for `PowerModelsDistribution._sc2br_impedance :: Tuple{Any}` in module `PowerModelsDistribution`
└ @ Base.Docs docs/Docs.jl:223


Before, you could import data from one of three sources:
- OpenDSS input files
- balanced matpower .m files, converted to unbalanced by PMs utilities
- unbalanced .m files, custom extension of the matpower format

We decided to drop support for the matpower extension. Instead, we decided to add a high-level, user-friendly data model directly in PMD. The OpenDSS parser will be updated to parse to this high-level data format. Furthermore, we defined a lot of utility functions which allow you to build your case study step-by-step, much like you can do it in OpenDSS.

## Build your own case

Start by creating an empty data model. For now this is still a regular Dict, but it might be wrapped in a struct in the future.

In [48]:
data_model = create_data_model()

Dict{String,Any} with 1 entry:
  "v_var_scalar" => 1000.0

First, we will add a voltage source to our network. Each component can be added by a similar syntax. First, you pass the data model you want to add the component to, and then you specify the properties of that component through keywords arguments. 
- **id**: This is an identifier for the component you create, which must be unique amongst components of the same type. This will be the key for the component dictionaries at the high-level (both for the data model and the solutions). The id can be anything (as in any type) you want, but if you want to save it as a json file, Strings or Integers are recommended.

Most values are entered in SI units (TODO: explain the scalar thing).

### Buses, linecodes and lines

In [49]:
add_bus!(data_model, id=:sourcebus, terminals=[:a, :b, :c, :n])
add_bus!(data_model, id=:loadbus,   terminals=[:a, :b, :c, :n])

Dict{String,Any} with 6 entries:
  "rg"        => Float64[]
  "grounded"  => Any[]
  "status"    => 1
  "terminals" => Symbol[:a, :b, :c, :n]
  "id"        => :loadbus
  "xg"        => Float64[]

In [50]:
add_voltage_source!(data_model, id=:source, bus=:sourcebus, vm=[0.23, 0.23, 0.23, 0], va=[0, -2*pi/3, 2*pi/3, 0])

Dict{String,Any} with 6 entries:
  "va"          => [0.0, -2.0944, 2.0944, 0.0]
  "status"      => 1
  "connections" => [1, 2, 3]
  "id"          => :source
  "vm"          => [0.23, 0.23, 0.23, 0.0]
  "bus"         => :sourcebus

In [51]:
add_linecode!(data_model, id=:lc_3w, rs=ones(3,3), xs=ones(3,3))

Dict{String,Any} with 7 entries:
  "b_fr" => [0.0 0.0 0.0; 0.0 0.0 0.0; 0.0 0.0 0.0]
  "rs"   => [1.0 1.0 1.0; 1.0 1.0 1.0; 1.0 1.0 1.0]
  "id"   => :lc_3w
  "xs"   => [1.0 1.0 1.0; 1.0 1.0 1.0; 1.0 1.0 1.0]
  "g_to" => [0.0 0.0 0.0; 0.0 0.0 0.0; 0.0 0.0 0.0]
  "b_to" => [0.0 0.0 0.0; 0.0 0.0 0.0; 0.0 0.0 0.0]
  "g_fr" => [0.0 0.0 0.0; 0.0 0.0 0.0; 0.0 0.0 0.0]

In [52]:
add_line!(data_model, id=:line1, linecode=:lc_3w, length=1.2, 
    f_bus=:sourcebus, f_connections=[:a, :b, :c], t_bus=:loadbus, t_connections=[:a, :b, :c],
)

Dict{String,Any} with 10 entries:
  "length"        => 1.2
  "t_connections" => Symbol[:a, :b, :c]
  "f_bus"         => :sourcebus
  "angmin"        => [-1.0472, -1.0472, -1.0472]
  "status"        => 1
  "angmax"        => [1.0472, 1.0472, 1.0472]
  "id"            => :line1
  "linecode"      => :lc_3w
  "t_bus"         => :loadbus
  "f_connections" => Symbol[:a, :b, :c]

### Loads

In [53]:
add_load!(data_model, id=:cp_y_2ph, pd=[0.1, 0.2], 
    bus=:loadbus, connections=[:a, :b, :n]
)

Dict{String,Any} with 8 entries:
  "status"        => 1
  "model"         => "constant_power"
  "connections"   => Symbol[:a, :b, :n]
  "id"            => :cp_y_2ph
  "qd"            => [0.0, 0.0, 0.0]
  "bus"           => :loadbus
  "pd"            => [0.1, 0.2]
  "configuration" => "wye"

In [54]:
add_load!(data_model, id=:cp_d_3ph, pd=[0.1, 0.2, 0.3], 
    bus=:loadbus, configuration="delta", connections=[:a, :b, :c]
)

Dict{String,Any} with 8 entries:
  "status"        => 1
  "model"         => "constant_power"
  "connections"   => Symbol[:a, :b, :c]
  "id"            => :cp_d_3ph
  "qd"            => [0.0, 0.0, 0.0]
  "bus"           => :loadbus
  "pd"            => [0.1, 0.2, 0.3]
  "configuration" => "delta"

In [55]:
add_load!(data_model, id=:cc_d_3ph, model="constant_current", pd_ref=[0.1, 0.2, 0.3], vnom=0.230*sqrt(3),
    bus=:loadbus, configuration="delta", connections=[:a, :b, :c]
)

Dict{String,Any} with 9 entries:
  "vnom"          => 0.398372
  "status"        => 1
  "model"         => "constant_current"
  "connections"   => Symbol[:a, :b, :c]
  "qd_ref"        => [0.0, 0.0, 0.0]
  "pd_ref"        => [0.1, 0.2, 0.3]
  "id"            => :cc_d_3ph
  "bus"           => :loadbus
  "configuration" => "delta"

Finally, we can check whether the data model is consistent. There are many built in checks to keep you from making mistakes. For example, it will check that
- all buses and terminals you used are actually defined;
- all required data model properties are present;
- all properties you have defined have the right type (real, boolean, array, ...);
- arrays and vectors have the right size (the length of the `connections` property usually implies the 'right' size);
- more specialized checks at the component level.

When you execute the following, these checks will print a descriptive error message if something is wrong. You can try this out by going back and entering something inconsistent, i.e. connecting a load to an undefined bus.


In [ ]:
check_data_model(data_model)